In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [ ]:
# Load train and test data
traindata = pd.read_csv('/kaggle/input/ucs-654-kaggle-hack-lab-exam-ii/train.csv')
print(traindata.shape)
traindata.head()
testdata = pd.read_csv('/kaggle/input/ucs-654-kaggle-hack-lab-exam-ii/test.csv')
print(testdata.shape)
testdata.head()

In [ ]:
# Separate features and target
x_train = traindata.drop(columns=['target'])
y_train = traindata['target']
x_test = testdata.drop(columns=['id'])

In [ ]:
# step 1: apply quantiletransformer

quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
x_train_transformed = quantile_transformer.fit_transform(x_train)
y_test_transformed = quantile_transformer.transform(x_test)

In [ ]:
# step 2: train the random forest regressor
rf_model = RandomForestRegressor(
    n_estimators=355,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    bootstrap=False,
    max_samples=None,
    random_state=42
)

rf_model.fit(x_train_transformed, y_train)

In [ ]:
# step 3: evaluate on training data
y_train_rf_pred = rf_model.predict(x_train_transformed)
rf_r2_score = r2_score(y_train, y_train_rf_pred)
print(f"random forest training r² score: {rf_r2_score}")

In [ ]:
# step 4: predict on test data
y_test_pred = rf_model.predict(y_test_transformed)

In [ ]:
# step 5: save predictions
output = pd.DataFrame({'id': testdata['id'], 'target': y_test_pred})
output.to_csv('submission.csv', index=False)
print("predictions saved to submission.csv")